In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import random
import re
import nltk
import operator

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')


In [2]:
train_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/train_super_featured.csv')
test_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/test_super_featured.csv')
train_data.head()

,id_original,keyword_original,location_original,text_original,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,...,super_stop_word_count,super_url_count,super_mean_word_length,super_char_count,super_punctuation_count,super_hashtag_count,super_mention_count,super_clean_text,kaggle_text,target_relabeled
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,1,#earthquake,NaN,1,0,...,6,0,4.38,69,1,1,0,deeds reason earthquake may allah forgive us,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1,NaN,NaN,0,0,...,0,0,4.57,38,1,0,0,forest fire near la ronge sask canada,Forest fire near La Ronge Sask . Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,3,NaN,NaN,0,0,...,11,0,5.09,133,3,0,0,residents asked shelter place notified officer...,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,2,#wildfires,NaN,1,0,...,1,0,7.12,65,2,1,0,people receive wildfires evacuation orders cal...,"13,000 people receive # wildfires evacuation ...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,2,#alaska #wildfires,NaN,2,0,...,6,0,4.73,87,2,2,0,got sent photo ruby alaska smoke wildfires pou...,Just got sent this photo from Ruby # Alaska a...,1


In [3]:
train_data['text_original'].fillna('', inplace=True)
train_data['clean_text'].fillna('', inplace=True)
train_data['super_clean_text'].fillna('', inplace=True)
train_data['kaggle_text'].fillna('', inplace=True)
train_data['semi_cleaned_text'].fillna('', inplace=True)
test_data['text_original'].fillna('', inplace=True)
test_data['clean_text'].fillna('', inplace=True)
test_data['super_clean_text'].fillna('', inplace=True)
test_data['kaggle_text'].fillna('', inplace=True)
test_data['semi_cleaned_text'].fillna('', inplace=True)

In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [5]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import time

from gensim.models.word2vec import Word2Vec
from glob import glob

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [8]:
target = train_data['target_relabeled'].values.tolist()
data = train_data['super_clean_text'].values.tolist()

In [9]:
x_train, x_test, y_train, y_test = \
    train_test_split(data,target, test_size=0.2)  # random split

In [10]:
tokenizer = Tokenizer()             #Tokenizer(num_words=5000) => 5000 words of the highest frequency
tokenizer.fit_on_texts(data)
tokenizer

In [11]:
print("len(tokenizer) :",len(list(tokenizer.word_index)))

len(tokenizer) : 14346


In [12]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)


In [13]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)


In [14]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [15]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

16

In [16]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9729410219361618

In [17]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)


In [18]:
#Zero is added before the values given in the padding operation.

print("x_train_tokens :",x_train_tokens[0])
print("x_train_pad :",x_train_pad[0])

x_train_tokens : [883, 884, 49, 245, 800, 678, 652, 64, 279, 385, 3839, 652, 1005, 279, 46, 49]
x_train_pad : [ 883  884   49  245  800  678  652   64  279  385 3839  652 1005  279
   46   49]


In [19]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)


In [20]:
print("x_train_pad.shape :",x_train_pad.shape)
print("x_train_pad.shape :",x_test_pad.shape)

x_train_pad.shape : (6090, 16)
x_train_pad.shape : (1523, 16)


In [21]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [22]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [23]:
x_train[800]


'calgary area tornado warnings end thunderstorms move eastward cbcca cbcca calgary area tornado warnings end u'

In [24]:
print(x_train_tokens[800])


[334, 222, 419, 1610, 194, 1542, 648, 6128, 4188, 4188, 334, 222, 419, 1610, 194, 1]


In [25]:
tokens_to_string(x_train_tokens[800])


'calgary area tornado warnings end thunderstorms move eastward cbcca cbcca calgary area tornado warnings end u'

In [26]:
glove_embeddings = {}
with open('../../../DataSets/TP2/glove.840B.300d.txt', encoding='UTF-8') as f:
    for line in f:
        values = line.replace("\n", "").split(" ")
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vec

In [27]:
def build_vocab(X):
    
    tweets = X.apply(lambda s: s.split()).values      
    vocab = {}
    
    for tweet in tweets:
        for word in tweet:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1                
    return vocab


def check_embeddings_coverage(X, embeddings):
    
    vocab = build_vocab(X)    
    
    covered = {}
    oov = {}    
    n_covered = 0
    n_oov = 0
    
    for word in vocab:
        try:
            covered[word] = embeddings[word]
            n_covered += vocab[word]
        except:
            oov[word] = vocab[word]
            n_oov += vocab[word]
            
    vocab_coverage = len(covered) / len(vocab)
    text_coverage = (n_covered / (n_covered + n_oov))
    
    sorted_oov = sorted(oov.items(), key=operator.itemgetter(1))[::-1]
    return sorted_oov, vocab_coverage, text_coverage

In [28]:
train_glove_oov, train_glove_vocab_coverage, train_glove_text_coverage = check_embeddings_coverage(train_data['text_original'], glove_embeddings)
test_glove_oov, test_glove_vocab_coverage, test_glove_text_coverage = check_embeddings_coverage(test_data['text_original'], glove_embeddings)
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Training Set text_original'.format(train_glove_vocab_coverage, train_glove_text_coverage))
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Test Set text_original'.format(test_glove_vocab_coverage, test_glove_text_coverage))

GloVe Embeddings cover 51.90% of vocabulary and 81.71% of text in Training Set text_original
GloVe Embeddings cover 57.12% of vocabulary and 81.00% of text in Test Set text_original


In [29]:
train_glove_oov, train_glove_vocab_coverage, train_glove_text_coverage = check_embeddings_coverage(train_data['clean_text'], glove_embeddings)
test_glove_oov, test_glove_vocab_coverage, test_glove_text_coverage = check_embeddings_coverage(test_data['clean_text'], glove_embeddings)
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Training Set clean_text'.format(train_glove_vocab_coverage, train_glove_text_coverage))
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Test Set clean_text'.format(test_glove_vocab_coverage, test_glove_text_coverage))

GloVe Embeddings cover 85.76% of vocabulary and 95.89% of text in Training Set clean_text
GloVe Embeddings cover 88.06% of vocabulary and 95.66% of text in Test Set clean_text


In [30]:
train_glove_oov, train_glove_vocab_coverage, train_glove_text_coverage = check_embeddings_coverage(train_data['kaggle_text'], glove_embeddings)
test_glove_oov, test_glove_vocab_coverage, test_glove_text_coverage = check_embeddings_coverage(test_data['kaggle_text'], glove_embeddings)
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Training Set kaggle_text'.format(train_glove_vocab_coverage, train_glove_text_coverage))
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Test Set kaggle_text'.format(test_glove_vocab_coverage, test_glove_text_coverage))

GloVe Embeddings cover 67.95% of vocabulary and 94.84% of text in Training Set kaggle_text
GloVe Embeddings cover 75.66% of vocabulary and 94.99% of text in Test Set kaggle_text


In [31]:
train_glove_oov, train_glove_vocab_coverage, train_glove_text_coverage = check_embeddings_coverage(train_data['super_clean_text'], glove_embeddings)
test_glove_oov, test_glove_vocab_coverage, test_glove_text_coverage = check_embeddings_coverage(test_data['super_clean_text'], glove_embeddings)
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Training Set super_clean_text'.format(train_glove_vocab_coverage, train_glove_text_coverage))
print('GloVe Embeddings cover {:.2%} of vocabulary and {:.2%} of text in Test Set super_clean_text'.format(test_glove_vocab_coverage, test_glove_text_coverage))

GloVe Embeddings cover 86.11% of vocabulary and 96.09% of text in Training Set super_clean_text
GloVe Embeddings cover 89.24% of vocabulary and 96.12% of text in Test Set super_clean_text


In [32]:
embedding_size = 300

In [33]:
print("x_test[0] :",x_test[40])
print("x_test_pad[0] :",x_test_pad[0])

x_test[0] : im going say new reddit policy changes think r conspiracy quarantined thats bad truth u
x_test_pad[0] : [   0    0    0    0    0    0    0  796  248 1944  742 3239 2207  284
 1319 6034]


In [34]:
num_words = len(list(tokenizer.word_index)) + 1


In [35]:
embedding_matrix = np.random.uniform(-1, 1, (num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    if i < num_words:
        embedding_vector = glove_embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [36]:
embedding_matrix.shape


(14347, 300)

In [37]:
embedding_matrix[3]


array([ 7.44820014e-02,  5.82930028e-01, -7.82339990e-01, -4.49389994e-01,
        1.93560004e-01,  1.68380007e-01,  2.84999996e-01, -4.01809990e-01,
       -7.54270032e-02,  1.16149998e+00, -5.73189974e-01,  2.72560000e-01,
       -1.46229997e-01, -4.52829987e-01,  7.65789986e-01,  1.09020002e-01,
       -2.05180004e-01,  1.00919998e+00, -3.18250000e-01,  4.14839983e-02,
       -5.20990007e-02,  1.25279993e-01,  6.20520003e-02, -4.52399999e-01,
       -2.21540004e-01, -2.54539996e-01, -3.68680000e-01, -3.57010007e-01,
        8.97620022e-01, -2.88789988e-01, -2.53369987e-01,  1.88620001e-01,
        1.54560000e-01,  1.91569999e-01,  1.16980001e-01,  1.17820002e-01,
       -2.23859996e-01,  1.61149994e-01, -1.17160000e-01, -1.72269996e-02,
       -1.79370001e-01, -5.06579995e-01, -4.70730007e-01,  2.95810014e-01,
       -2.33830005e-01,  5.29030025e-01,  2.74870008e-01, -8.10769975e-01,
       -2.54729986e-01, -2.86720008e-01, -1.02229998e-01, -1.10290004e-02,
       -1.66869998e-01, -

In [38]:
sequence_length = max_tokens
vocabulary_size = num_words
embedding_dim = embedding_size
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

In [39]:
epochs = 5
batch_size = 30

In [40]:
y_train2 = np.array([])
y_test2 = np.array([])
for i in y_train:
    y_train2 = np.append(y_train2, i)
for i in y_test:
    y_test2 =np.append(y_test2, i)

In [49]:
#CNN architecture

num_classes = 2

#Training params
batch_size = 512 
num_epochs = 5

#Model parameters
num_filters = 32  # görüntünün boyutu mesela 512*512
embed_dim = embedding_size 
weight_decay = 1e-2

print("training CNN ...")
model = Sequential()

#Model add word2vec embedding

model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    weights= [embedding_matrix],
                    input_length=max_tokens,        
                    trainable=True,              #the layer is trained
                    name='embedding_layer'))
model.add(Conv1D(num_filters, 7, activation='tanh', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='tanh', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='softmax'))  #multi-label (k-hot encoding)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()



#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=8, verbose=1)
callbacks_list = [early_stopping]


hist = model.fit(x_train_pad, y_train2, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

training CNN ...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 16, 300)           4304100   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 16, 32)            67232     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 8, 32)             7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 3

In [50]:
predicted = model.predict(x_test_pad)
predicted =np.argmax(predicted, axis=1) 
np.mean(predicted == y_test2)

0.8273145108338805

In [44]:
x_final_test_tokens = tokenizer.texts_to_sequences(test_data['super_clean_text'])
x_final_test_pad = pad_sequences(x_final_test_tokens, maxlen=max_tokens)
predicted = model.predict(x_final_test_pad)
predicted =np.argmax(predicted, axis=1) 

In [46]:
test_data['target'] = predicted
test_data[['id_original', 'target']].rename(columns={'id_original': 'id'}).to_csv('~/Documents/Datos/DataSets/TP2/res_TF_GloVe.2.csv', index=False)